In [1]:
import torch
from torch.utils.data import DataLoader
from functools import partial
import transformers
from tqdm import tqdm
import pickle

In [2]:
import pickle
with open('y_pred_msvdqa.pkl', 'rb') as f:
    y_pred = pickle.load(f)

with open('y_real_msvdqa.pkl', 'rb') as f:
    y_real = pickle.load(f)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [7]:
len(y_pred)

800

In [6]:
y_real[:30]

['No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'Yes']

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


In [3]:
import json

with open('val_qa.json') as f:
    dataset = json.load(f)
    print(len(dataset))

6415


In [4]:
dataset[1]

{'answer': 'man',
 'id': 30934,
 'question': 'who pours a seasoning liquid from a plastic container over chicken pieces placed in a plastic pouch?',
 'video_id': 1201}

In [5]:
from collections import defaultdict

id2video_mapper = defaultdict()

with open("youtube_mapping.txt") as file:
    lines = [line.rstrip() for line in file]

for line in lines:
    key, value = line.split()
    value = value.split('vid')[1]
    id2video_mapper[value] = key


In [6]:

class MSVDQaDataset(torch.utils.data.Dataset):

    def __init__(self, module_path, dataset, mapper, task_type = 'qa', question_prompt = " Answer the question using a single word or a short phrase with multiple words."):
        
        self.data_list = []
        self.question_prompt = question_prompt

        self.movie_dir = module_path

        for elem in dataset:
            full_video_path = module_path + '/eval/YouTubeClips' +'/' + mapper[str(elem['video_id'])] +".avi"
            self.data_list.append({'full_video_path':full_video_path, 'question':elem['question'], 'answer':elem['answer']})
            

        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data_item = self.data_list[idx]
        video_path = data_item['full_video_path']
        
        if not os.path.exists(video_path):
            print (video_path)
            print(f"Warning: Video file not found at {video_path}, skipping this item.")
            return None  
    

        question = data_item['question']
        answer = data_item['answer']
    
        return {
            'question': question + self.question_prompt,
            'video_path': video_path,
            #'pixel_values': pixel_values,
            'answer': answer.capitalize(),
        }
    

In [7]:
msvdq = MSVDQaDataset(module_path, dataset, id2video_mapper, task_type = 'qa') 

## Inference

In [8]:
import os
import sys
import torch
import numpy as np
import random
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.cluster_size', 'model.motion_tokenizer.quantize.embedding.embed_avg', 'model.motion_tokenizer.quantize.embedding.initted', 'model.motion_tokenizer.quantize.embedding.cluster_size']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16


In [9]:
video_path = '/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/bQJQGoJF7_k_162_169.avi'
prompt = "who pours marinade in a bag of chicken? Answer the question using a single word or phrase."

output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be

Man


In [10]:
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_path, use_fast=False, padding_side='left')
tokenizer.pad_token = tokenizer.unk_token

In [11]:
def collate_fn(batches, tokenizer):
    
    questions = [_['question'] for _ in batches]
    video_path = [_['video_path'] for _ in batches]
    answer = [_['answer'] for _ in batches]
    
    return questions, video_path, answer

dataloader = torch.utils.data.DataLoader(
        dataset=msvdq,
        batch_size=1,
        num_workers=1,
        pin_memory=True,
        drop_last=False,
        collate_fn=partial(collate_fn, tokenizer=tokenizer)
    )

iterator = iter(dataloader)
first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]


print(question)
print(answer)
print(video_path)


first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]


print(question)
print(answer)
print(video_path)


first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]


print(question)
print(answer)
print(video_path)


['who pours liquid from a plastic container into a ziploc bag containing meat pieces? Answer the question using a single word or a short phrase with multiple words.']
['Someone']
['/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/bQJQGoJF7_k_162_169.avi']
['who pours a seasoning liquid from a plastic container over chicken pieces placed in a plastic pouch? Answer the question using a single word or a short phrase with multiple words.']
['Man']
['/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/bQJQGoJF7_k_162_169.avi']
['who pours marinade in a bag of chicken? Answer the question using a single word or a short phrase with multiple words.']
['Person']
['/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/bQJQGoJF7_k_162_169.avi']


In [ ]:
y_pred = []
y_real = []

progress_bar = tqdm(
        dataloader, total=len(dataloader), desc=f"Epoch 1"
    )

for step, batch in enumerate(progress_bar, start=1):
    question = batch[0]
    video_path = batch[1]
    answer = batch[2]

    # print(pixel_values.size())
    # print(question)
    # print(answer)
    # print(num_patches_list)
    # print(task_type)
    
    # with autocast():
        # 执行推理
    outputs = runner({"video": video_path[0], "text_input": question[0]}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]

    y_pred.append(outputs.strip())
    y_real.append(answer[0].strip())
    if (step%100 == 0):
        with open('y_pred_msvdqa.pkl', 'wb') as f:
            pickle.dump(y_pred, f)
        with open('y_real_msvdqa.pkl', 'wb') as f:
            pickle.dump(y_real, f)
        print("="*20 + "question" + "="*20)
        print (question[0])
        print("="*20 + "output" + "="*20)
        print(outputs)
        print("="*20 + "real answers" + "="*20)
        print(answer[0], flush = True)

Epoch 1:   2%|▏         | 99/6415 [04:26<2:27:03,  1.40s/it]

====================question====================
who made a great catch? Answer the question using a single word or a short phrase with multiple words.
====================output====================
Man
====================real answers====================
Baseball


Epoch 1:  16%|█▌        | 999/6415 [39:59<2:57:50,  1.97s/it]

====================question====================
who is stroking a baby beaver? Answer the question using a single word or a short phrase with multiple words.
====================output====================
Man
====================real answers====================
Woman


Epoch 1:  17%|█▋        | 1099/6415 [42:42<2:04:05,  1.40s/it]

...

Epoch 1:  98%|█████████▊| 6299/6415 [4:00:39<01:48,  1.07it/s]

====================question====================
what is putting a lizard into a box? Answer the question using a single word or a short phrase with multiple words.
====================output====================
Person
====================real answers====================
Someone


Epoch 1: 100%|█████████▉| 6399/6415 [4:03:05<00:17,  1.08s/it]

====================question====================
what is a man sliding across with a shopping cart? Answer the question using a single word or a short phrase with multiple words.
====================output====================
No idea
====================real answers====================
Floor


Epoch 1: 100%|██████████| 6415/6415 [4:03:26<00:00,  2.28s/it]


In [13]:
y_pred[:30]

['Man',
 'Guy',
 'Man',
 'Chef',
 'Guy',
 'Man',
 'Chef',
 'Chef',
 'Butcher',
 'Chef',
 'Unknown',
 'Marinade',
 'Woman',
 'Man',
 'Sauce',
 'Person',
 'A cook pours sauce on slices of meat.',
 'Raw chicken',
 'Bag',
 'Plastic bag',
 "The video doesn't show what the marinade is made from.",
 'Chicken',
 'Tray',
 'Spices',
 'Sauce',
 'Sauce',
 'Seasoning',
 'Seasoning',
 'Chicken',
 'Ziplock']

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [14]:
y_real[:30]

['Someone',
 'Man',
 'Person',
 'Man',
 'Man',
 'Man',
 'Man',
 'Cook',
 'Man',
 'Person',
 'Person',
 'Seasoning',
 'Man',
 'Man',
 'Sauce',
 'Man',
 'Marinade',
 'Chicken',
 'Bag',
 'Bag',
 'Bowl',
 'Chicken',
 'Bag',
 'Marinade',
 'Sauce',
 'Marinade',
 'Marinade',
 'Sauce',
 'Meat',
 'Bag']

In [15]:
result = [1 if i == j else 0 for i, j in zip(y_real, y_pred)]

print("VideoLAVIT_MSVDQA, accuracy is: " + str(sum(result) / len(result)))

VideoLAVIT_MSVDQA, accuracy is: 0.27030397505845677


## Evaluation by Gpt-3

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = "YOUR_API_KEY"

# Define the test cases
test_cases = [
    {"input": "What is the capital of France?", "expected_output": "Paris"},
    {"input": "What is 2 + 2?", "expected_output": "4"},
    {"input": "Explain photosynthesis briefly.", "expected_output": "Photosynthesis is the process by which plants convert sunlight into energy."},
]

# Validate the model
for test in test_cases:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": test["input"]}]
    )
    output = response['choices'][0]['message']['content'].strip()
    print(f"Input: {test['input']}")
    print(f"Expected: {test['expected_output']}")
    print(f"Output: {output}")
    print(f"Match: {output == test['expected_output']}")
    print("-" * 30)